In [1]:
import os
import importlib

if not os.path.exists('./bearing-conf-interval'):
    !git clone https://github.com/Gvalhca/bearing-conf-interval.git
mlab_func = importlib.import_module("bearing-conf-interval.plots.mlab_func")

In [2]:
!pip install ipympl
!pip install gdown
!apt install rar -y

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [3]:
import pathlib

# pathlib.Path("./data").mkdir(parents=True, exist_ok=True)
# !gdown --fuzzy https://drive.google.com/file/d/1dJatTfakPEQJhHPKDPgk6xRCE7fNfxpS/view?usp=sharing
# !gdown --fuzzy https://drive.google.com/file/d/1oX408ggrvAWd-AJvlTnzgmt7W25bMjUZ/view?usp=sharing

In [4]:
! for rar in./ *.rar; do rar e -y "$rar"./ data /; done

/bin/bash: -c: line 0: syntax error near unexpected token `in./'
/bin/bash: -c: line 0: ` for rar in./ *.rar; do rar e -y "$rar"./ data /; done'


In [5]:
%matplotlib qt
# from google.colab import output
# output.enable_custom_widget_manager()
import os.path
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy.stats
from typing import Tuple

In [6]:
class BearingPlot:
    def __init__(self, ds_name: str, data_path: str, reb_code='Ax 1_1', sensor_type='S1', dataset_name='IMS'):
        ## Initial Parameters
        if dataset_name == 'IMS':
            self.Fs = 20000
            self.N = 20480
            self.t = np.linspace(0, self.N / self.Fs, self.N)
            self.ShaftSpeed = 2000 / 60
            self.BPFI = 8.908 * self.ShaftSpeed
            self.BPFO = 7.092 * self.ShaftSpeed
            self.BSF = 4.197 * self.ShaftSpeed
            self.data_path = os.path.join(data_path, '2nd_test-4sensor.mat')
            # self.data_path = data_path
        else:
            self.Fs = 25600
            self.N = 2560
            self.t = np.linspace(0, 0.1, self.N)
            self.REBcode = reb_code
            self.ShaftSpeed = 1800 / 60
            self.BPFI = 7.389 * self.ShaftSpeed
            self.BPFO = 5.611 * self.ShaftSpeed
            self.BSF = 3.589 * self.ShaftSpeed
            self.data_path = os.path.join(data_path, self.REBcode + '.mat')
            # self.data_path = data_path
        self.dataset_name = dataset_name
        self._load_data(sensor_type)

    @property
    def xlim(self):
        return self.Acc.shape[-1] - 1

    def _load_data(self, sensor_type):
        ## Load data
        data_bearing = scipy.io.loadmat(self.data_path)

        if self.dataset_name == 'IMS':
            sensor_type = sensor_type
        else:
            sensor_type = 'ACCx'
        self.VibData = data_bearing[sensor_type]
        self.Life = self.VibData[0, :]
        self.Acc = self.VibData[1:, :]

    def _draw_freq_plots_at_point(self, x):
        self._i += 1
        a = int(np.floor(self._i / 4))
        c = np.abs(2.1 - a)
        b = self._i - a * 4
        self._rms_ax.scatter(x, self._peak[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)
        self._kurt_ax.scatter(x, self._kurt[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)
        self._cf_ax.scatter(x, self._CF[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)

        ## Time wave and Spectrum presentation
        current_signal = self.Acc[:, x]
        self._amplitude_ax.plot(self.t, current_signal, color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=0.5)
        self._amplitude_ax.grid(True)
        self._amplitude_ax.set_xlabel('t (s)')
        self._amplitude_ax.set_ylabel('Amplitude (g)')

        Y, f = mlab_func.spect_gen(current_signal, self.Fs)
        self._spect_gen_ax.plot(f, Y, color=np.array([1 - b / 4, b / 4, 1 - b / 4]))
        self._spect_gen_ax.set_title('FFT of Vibration signal')
        self._spect_gen_ax.set_xlabel('Frequency (Hz)')
        self._spect_gen_ax.set_ylabel('Amplitude (g)')
        self._spect_gen_ax.set_xlim(np.array([0, 1000]))

        self._freq_scaled_ax.plot(f, Y, color=np.array([1 - b / 4, b / 4, 1 - b / 4]))
        self._freq_scaled_ax.set_title('FFT of Vibration signal')
        self._freq_scaled_ax.set_xlabel('Frequency (Hz)')
        self._freq_scaled_ax.set_ylabel('Amplitude (g)')
        self._freq_scaled_ax.grid(True)
        self._freq_scaled_ax.set_xlim(np.array([0, self.Fs / 2.5]))

        #Plot the characteristic frequencies
        x_bounds = self._spect_gen_ax.get_xlim()
        self._spect_gen_ax.axvline(self.BPFI, c='y', ls='--', label='BPFI')
        self._spect_gen_ax.annotate(s='BPFI', xy=(((self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='y')
        self._spect_gen_ax.axvline(2 * self.BPFI, c='y', ls='--', label='2xBPFI')
        self._spect_gen_ax.annotate(s='2xBPFI',
                                    xy=(((2 * self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='y')
        self._spect_gen_ax.axvline(3 * self.BPFI, c='y', ls='--', label='3xBPFI')
        self._spect_gen_ax.annotate(s='3xBPFI',
                                    xy=(((3 * self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='y')
        self._spect_gen_ax.axvline(self.BSF, c='g', ls='--', label='BSF')
        self._spect_gen_ax.annotate(s='BSF', xy=(((self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='g')
        self._spect_gen_ax.axvline(2 * self.BSF, c='g', ls='--', label='2xBSF')
        self._spect_gen_ax.annotate(s='2xBSF', xy=(((2 * self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='g')
        self._spect_gen_ax.axvline(3 * self.BSF, c='g', ls='--', label='3xBSF')
        self._spect_gen_ax.annotate(s='3xBSF', xy=(((3 * self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='g')
        self._spect_gen_ax.axvline(self.BPFO, c='r', ls='--', label='BPFO')
        self._spect_gen_ax.annotate(s='BPFO', xy=(((self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='r')
        self._spect_gen_ax.axvline(2 * self.BPFO, c='r', ls='--', label='2xBPFO')
        self._spect_gen_ax.annotate(s='2xBPFO',
                                    xy=(((2 * self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='r')
        self._spect_gen_ax.axvline(3 * self.BPFO, c='r', ls='--', label='3xBPFO')
        self._spect_gen_ax.annotate(s='3xBPFO',
                                    xy=(((3 * self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                    xycoords='axes fraction', verticalalignment='top', horizontalalignment='left',
                                    rotation=270,
                                    fontsize=12, color='r')

    def draw_signal_plots(self, signal_nums):
        ## TDF Calculation and presentation
        # RMS
        self.RMS = np.sqrt(np.mean(self.Acc ** 2, axis=0))
        fig = plt.figure(self.dataset_name, figsize=(20, 10))
        fig.clear()
        self._rms_ax = plt.subplot(3, 2, 1)
        plt.ion()
        plt.plot(self.RMS, 'y')
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('RMS & Peak (g)')

        # Peak
        self._peak = np.max(np.abs(self.Acc), axis=0)
        plt.plot(self._peak, 'r')
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('RMS & Peak (g)')
        plt.legend(['RMS', 'Peak'], loc='best', prop={'size': 12})

        # Kurtosis
        self._kurt = scipy.stats.kurtosis(self.Acc)
        self._kurt_ax = plt.subplot(3, 2, 3)
        plt.plot(self._kurt)
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('Kurtosis')

        #Crest Factor
        self._CF = self._peak / self.RMS
        self._cf_ax = plt.subplot(3, 2, 5)
        plt.plot(self._CF)
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('CF')

        self._amplitude_ax = plt.subplot(3, 2, 2)
        self._spect_gen_ax = plt.subplot(3, 2, 4)
        self._freq_scaled_ax = plt.subplot(3, 2, 6)

        self._i = 0
        if isinstance(signal_nums, int):
            self._draw_freq_plots_at_point(signal_nums)
            legends = ['RMS', 'Peak'] + [str(signal_nums)]
        else:
            for x in signal_nums:
                self._draw_freq_plots_at_point(x)
            legends = ['RMS', 'Peak'] + list(signal_nums)

        self._rms_ax.legend(legends)
        fig.tight_layout()
        fig.canvas.draw()
        plt.show()

In [72]:
dataset_name = 'IMS'  #@param ['IMS', 'PRONOSTIA']
data_path = '/ssd/Code/iran-bearing/data'  #@param {type:"string"}
# data_path = '/content/data/Pronostia data/Ax 1_1.mat'

if dataset_name == 'PRONOSTIA':
    #@markdown ---
    #@markdown ##### Name of PRONOSTIA .mat file:
    reb_code = 'Ax 1_3'  #@param {type:"string"}
    sensor_type = None
else:
    reb_code = None
    #@markdown ---
    #@markdown ##### Sensor type in IMS set:
    sensor_type = 'S1'  #@param ['S1', 'S2', 'S3', 'S4']
#@markdown ---
#@markdown ##### Enter signal numbers, e.g. (0, 10, 100, 200,):
# signals = (0, 10, 100, 200, 500, 750)
signals = (0, 2500)  #@param {type:"raw"}

#@markdown ---
#@markdown ##### Use one point interactive mode with slider?
one_point_mode = True  #@param {type:"boolean"}

import ipywidgets as widgets

bearing_plot = BearingPlot(ds_name=dataset_name, data_path=data_path, reb_code=reb_code, dataset_name=dataset_name)

if one_point_mode:
    widgets.interact(bearing_plot.draw_signal_plots,
                     signal_nums=widgets.IntSlider(value=0, min=0, max=bearing_plot.xlim, step=1))
else:
    bearing_plot.draw_signal_plots(signals)


interactive(children=(IntSlider(value=0, description='signal_nums', max=983), Output()), _dom_classes=('widget…

In [73]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
f = KalmanFilter(dim_x=2, dim_z=1)
f.x = np.array([2., 0.])
f.F = np.array([[1.,1.],
                [0.,1.]])
f.H = np.array([[1.,0.]])
f.P = np.array([[1000.,    0.],
               [   0., 1000.] ])
f.R = 5
f.Q = Q_discrete_white_noise(dim=2, dt=1, var=0.13)
# f.Q = 0.5 ** 2

estimates, likelihoods, log_likelihoods, stds = ([] for i in range(4))
conf_intervals = []
percentile = 1.96

for rms_measure in bearing_plot.RMS:
    f.predict()
    f.update(rms_measure)
    std = np.sqrt(np.diag(f.P))[0]
    ci = percentile * std / np.sqrt(len(stds) + 1)
    estimates.append(f.x[0])
    likelihoods.append(f.likelihood)
    log_likelihoods.append(f.log_likelihood)
    stds.append(std)
    conf_intervals.append(ci)

conf_intervals = np.array(conf_intervals)
x_data = np.arange(bearing_plot.RMS.shape[0])
bound_lower = estimates - conf_intervals
bound_upper = estimates + conf_intervals

fig_kf = plt.figure('Kalman Filter', figsize=(20, 10))
plt.plot(bearing_plot.RMS, '-r')
plt.plot(estimates, '--bo', markersize=2, alpha=0.7)
plt.grid(True)
plt.legend(['RMS', 'Kalman Filter'])
plt.fill_between(x_data, bound_lower, bound_upper, color='b', alpha=.15)
plt.show()

In [42]:
# from scipy.optimize import curve_fit
# import matplotlib.pylab as plt
# import numpy as np
#
# # model function
# func = lambda x, a, b: a * (1 / (x**2)) + b
#
# # approximating OP points
# n_ypoints = 7
# x_data = np.linspace(70, 190, n_ypoints)
#
# # approximating the original scatter in Y-data
# n_nested_points = 1000
# point_errors = 50
# y_data = [func(x, 4e6, -100) + np.random.normal(x, point_errors,
#                                                 n_nested_points) for x in x_data]
#
# # averages and dispersion of data
# y_means = np.array(y_data).mean(axis = 1)
# y_spread = np.array(y_data).std(axis = 1)
#
# best_fit_ab, covar = curve_fit(func, x_data, y_means,
#                                sigma = y_spread,
#                                absolute_sigma = True)
# sigma_ab = np.sqrt(np.diagonal(covar))
#
# from uncertainties import ufloat
# a = ufloat(best_fit_ab[0], sigma_ab[0])
# b = ufloat(best_fit_ab[1], sigma_ab[1])
# text_res = "Best fit parameters:\na = {}\nb = {}".format(a, b)
# print(text_res)
#
# # plotting the unaveraged data
# flier_kwargs = dict(marker = 'o', markerfacecolor = 'silver',
#                     markersize = 3, alpha=0.7)
# line_kwargs = dict(color = 'k', linewidth = 1)
# bp = plt.boxplot(y_data, positions = x_data,
#                  capprops = line_kwargs,
#                  boxprops = line_kwargs,
#                  whiskerprops = line_kwargs,
#                  medianprops = line_kwargs,
#                  flierprops = flier_kwargs,
#                  widths = 5,
#                  manage_ticks = False)
# # plotting the averaged data with calculated dispersion
# #plt.scatter(x_data, y_means, facecolor = 'silver', alpha = 1)
# #plt.errorbar(x_data, y_means, y_spread, fmt = 'none', ecolor = 'black')
#
# # plotting the model
# hires_x = np.linspace(50, 190, 100)
# plt.plot(hires_x, func(hires_x, *best_fit_ab), 'black')
# bound_upper = func(hires_x, *(best_fit_ab + sigma_ab))
# bound_lower = func(hires_x, *(best_fit_ab - sigma_ab))
# # plotting the confidence intervals
# plt.fill_between(hires_x, bound_lower, bound_upper,
#                  color = 'black', alpha = 0.15)
# plt.text(140, 800, text_res)
# plt.xlim(40, 200)
# plt.ylim(0, 1000)
# plt.show()

Best fit parameters:
a = (3.36+/-0.32)e+06
b = 84+/-32


(1000,)
